# Preprocessing 2:

Lo scopo di questa pipeline è la valutazione degli errori e poi l'indirizzamento verso un motodo per studiare i dati.
Per prima cosa misureremo gli errori derivanti dalla scelta di una particolare baseline.
Di seguito si vedrà usato il modello di Van't Hoff che prende in considerazione solo 2 stati accessibili:
- Folded;
- Unfolded.

In [1]:
# importo dati e librerie
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

folder_path = r"/Users/marco/Desktop/TESI/LOCAL_P/tel23/DSC/ci_puliti"

# Inizializza dizionario
ci_finali_data = {}

# Caricamento
for filename in os.listdir(folder_path):
    if filename.endswith(".csv") and filename.startswith("c"):
        path = os.path.join(folder_path, filename)
        df = pd.read_csv(path, sep="\t")
        ci_finali_data[filename.replace(".csv", "")] = df

# Mostra le anteprime (primi 5 file)
for nome, df in list(ci_finali_data.items())[:5]:
    print(f"\n--- {nome} ---")
    display(df.head())


--- c3_cubica ---


,Temperatura,Cp_corretto,Cp_baseline,Cp_finale_cubica
0,41.937,-2004.326385,-2165.354337,161.027952
1,41.979,-2004.818889,-2160.573354,155.754465
2,42.021,-2005.311393,-2155.805949,150.494555
3,42.064,-2015.451104,-2150.939082,135.487978
4,42.106,-2010.980154,-2146.199105,135.218951



--- c3_gradino ---


,Temperatura,Cp_corretto,Cp_baseline,Cp_finale_gradino
0,46.117,-1800.533137,-1618.306097,-182.227039
1,46.159,-1798.235380,-1618.306097,-179.929282
2,46.201,-1795.937623,-1618.306097,-177.631525
3,46.242,-1792.424733,-1618.306097,-174.118636
4,46.284,-1792.084396,-1618.306097,-173.778299



--- c3_lineare ---


,Temperatura,Cp_corretto,Cp_baseline,Cp_finale_lineare
0,46.117,-1800.533137,-1510.933473,-289.599664
1,46.159,-1798.235380,-1510.932663,-287.302716
2,46.201,-1795.937623,-1510.931854,-285.005769
3,46.242,-1792.424733,-1510.931064,-281.493669
4,46.284,-1792.084396,-1510.930254,-281.154142



--- c3_sigmoide ---


,Temperatura,Cp_corretto,Cp_baseline,Cp_finale_sigmoide
0,49.459,-1618.877387,-1618.264906,-0.612481
1,49.502,-1618.297206,-1618.265433,-0.031772
2,49.544,-1605.803680,-1618.265943,12.462262
3,49.586,-1593.310154,-1618.266445,24.956291
4,49.627,-1590.256951,-1618.266930,28.009979



--- c5_cubica ---


,Temperatura,Cp_corretto,Cp_baseline,Cp_finale_cubica
0,42.523,-2222.032297,-2469.911126,247.878829
1,42.564,-2220.147232,-2463.588775,243.441543
2,42.608,-2222.026675,-2456.823935,234.797260
3,42.649,-2220.017146,-2450.539061,230.521915
4,42.691,-2217.781671,-2444.119607,226.337936


## Studio degli errori:

Per ciascun campione è stato integrato il calore specifico corretto, usando:

$$
\Delta H = \int_{T_{min}}^{T_{max}} C_p(T) \, dT
$$

La procedura è ripetuta per 4 diversi modelli di baseline: lineare, cubica, a gradino e sigmoide.

Per stimare l'incertezza legata alla scelta del modello di baseline, viene calcolata la **deviazione standard** dei valori di ΔH ottenuti con le diverse baseline, assumendo che il contributo maggiore all'errore sia proprio legato a questa scelta.


In [2]:
records = []  # (Campione, Baseline, DeltaH)
for name, df in ci_finali_data.items():
    campione, baseline = name.split("_", 1)

    # Trova colonna con Cp finale
    col_cp = [c for c in df.columns if c.startswith("Cp_finale")][0]
    T = df["Temperatura"].values
    Cp = df[col_cp].values

    delta_H = np.trapezoid(Cp, T)
    records.append((campione, baseline, delta_H))

df_DH = pd.DataFrame(records, columns=["Campione", "Baseline", "DeltaH"])

# === 3. Calcolo errore tra baseline ===========
df_std = df_DH.groupby("Campione")["DeltaH"].std().rename("Errore_baseline")
df_DH = df_DH.merge(df_std, on="Campione").sort_values(["Campione", "Baseline"])

# === 4. Output ================================
pd.set_option("display.precision", 6)
print(df_DH.to_string(index=False))

Campione Baseline       DeltaH  Errore_baseline
      c3   cubica 34159.332539      5971.075017
      c3  gradino 46626.665928      5971.075017
      c3  lineare 42008.921246      5971.075017
      c3 sigmoide 46987.293683      5971.075017
      c5   cubica 34292.534704      6481.921626
      c5  gradino 47446.113983      6481.921626
      c5  lineare 45683.461325      6481.921626
      c5 sigmoide 48141.133524      6481.921626
      c7   cubica 30715.166830      5665.014682
      c7  gradino 40485.064252      5665.014682
      c7  lineare 43458.259125      5665.014682
      c7 sigmoide 41357.389694      5665.014682


## Valutazione della cooperatività della transizione termica

Per determinare se il processo di unfolding segue un modello a due stati cooperativo oppure coinvolge stati intermedi, si confrontano due entalpie:

- **ΔH<sub>cal</sub>**: entalpia calorimetrica, ottenuta integrando la curva di capacità termica:
  
  $$
  \Delta H_{cal} = \int C_p \, dT
  $$

- **ΔH<sub>vH</sub>**: entalpia di van’t Hoff, stimata assumendo una transizione a due stati:
  
  $$
  \Delta H_{vH} = \frac{4RT_m^2 \cdot C_p(T_m)}{\Delta H_{cal}}
  $$

dove:
- $ R $ è la costante dei gas (1.987 cal/mol·K),
- $ T_m $ è la temperatura trovata grazie al fit della curva $ C_p $ con la funzione di Van't Hoff,
- $ C_p(T_m) $ è il valore di capacità termica al picco.

### Interpretazione del rapporto:

Si calcola:

$$
\frac{\Delta H_{vH}}{\Delta H_{cal}}
$$

E si valuta secondo le soglie seguenti:

- **> 0.95** → Transizione altamente cooperativa (modello a due stati)
- **0.85 – 0.95** → Transizione con deviazioni moderate dal modello a due stati
- **< 0.85** → Transizione non cooperativa, con presenza di più stati intermedi

Questo criterio aiuta a stabilire se è necessario applicare un'analisi più complessa (es. deconvoluzione Freire–Biltonen) o se un semplice modello a due stati è sufficiente.


In [3]:
def interpreta_cooperativita(ratio):
    if np.isnan(ratio):
        return "Valore non disponibile"
    elif ratio < 0.85:
        return "Transizione non cooperativa"
    elif ratio < 0.95:
        return "Moderata deviazione dal modello a due stati"
    else:
        return "Alta cooperatività (modello a due stati)"


### Stima della temperatura di melting $ T_m $ mediante fit analitico

Per ottenere una stima accurata della temperatura di melting $ T_m $, il tratto di picco del segnale $ C_p(T) $ è stato fittato utilizzando una forma semplificata della **funzione di Van’t Hoff**, particolarmente adatta a descrivere transizioni cooperative tra due stati.

La funzione utilizzata per il fit è:

$$
C_p(T) = \frac{A}{\cosh^2\left( \frac{T - T_m}{b} \right)}
$$

dove:
- $ A $ è un parametro proporzionale all’altezza massima del picco,
- $ T_m $ è la temperatura di melting (valore centrale della transizione),
- $ b $ è un parametro legato alla larghezza della transizione.

Questa funzione deriva da una formulazione del calore specifico per una transizione a due stati ed è in grado di catturare la forma tipica dei picchi di unfolding proteico o nucleico.

Il fit è stato effettuato per ciascun campione, e la stima di $ T_m $ ottenuta è stata poi utilizzata nel calcolo dell’entalpia di Van’t Hoff.


In [4]:
from scipy.optimize import curve_fit
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np

# Funzione di van’t Hoff
def vanthoff_model(T, A, Tm, b):
    return A / np.cosh((T - Tm) / b)**2

# Dizionario per Tm stimati
Tm_dict = {}         # key = "c3_lineare", value = Tm in Kelvin
fit_param_dict = {}  # key = "c3_lineare", value = (A, Tm, b)
fit_sigma_dict = {}  

# === Calcolo batch dei Tm stimati ===
for name, df in ci_finali_data.items():

    T = df["Temperatura"].values
    Cp = df[[c for c in df.columns if c.startswith("Cp_finale")][0]].values

    A0, Tm0, b0 = Cp.max(), T[np.argmax(Cp)], 2.0
    popt, pcov = curve_fit(vanthoff_model, T, Cp, p0=[A0, Tm0, b0])
    A_fit, Tm_fit, b_fit = popt
    sigma_Tm = np.sqrt(pcov[1, 1]) if pcov is not None else np.nan   # errore 1 σ

    fit_param_dict[name] = (A_fit, Tm_fit, b_fit)
    fit_sigma_dict[name] = sigma_Tm           #  <<--- salva l’errore
    Tm_dict[name] = Tm_fit

# Estrai nomi validi
campioni = sorted(set(name.split("_")[0] for name in Tm_dict.keys()))
baselines = sorted(set(name.split("_")[1] for name in Tm_dict.keys()))

# Funzione interattiva (usa solo i fit salvati)
def plot_fit_interattivo(campione, baseline):
    key = f"{campione}_{baseline}"

    if key not in ci_finali_data or key not in fit_param_dict:
        print(f"Fit non disponibile per {key}")
        return

    df = ci_finali_data[key]
    T = df["Temperatura"].values
    Cp = df[[c for c in df.columns if c.startswith("Cp_finale")][0]].values

    A_fit, Tm_fit, b_fit = fit_param_dict[key]
    T_fit = np.linspace(T.min(), T.max(), 500)
    Cp_fit = vanthoff_model(T_fit, A_fit, Tm_fit, b_fit)

    plt.figure(figsize=(8, 5))
    plt.plot(T, Cp, 'o', label="Dati")
    plt.plot(T_fit, Cp_fit, '-', label=f"Fit van’t Hoff\nTm ≈ {Tm_fit:.2f} °C")
    plt.xlabel("Temperatura (°C)")
    plt.ylabel("Cp")
    plt.title(f"Fit van’t Hoff – {key}")
    plt.legend()
    plt.grid(True)
    plt.show()

# Widget interattivi
campione_widget = widgets.Dropdown(options=campioni, description="Campione")
baseline_widget = widgets.Dropdown(options=baselines, description="Baseline")

ui = widgets.HBox([campione_widget, baseline_widget])
out = widgets.interactive_output(plot_fit_interattivo, {"campione": campione_widget, "baseline": baseline_widget})

display(ui, out)


Output()

In [5]:
# --- all’inizio, hai già salvato i parametri del fit ----------
# fit_param_dict[name] = (A_fit, Tm_fit, b_fit)

R = 1.987  # cal/mol·K
records = []

for _, row in df_DH.iterrows():
    campione  = row["Campione"]
    baseline  = row["Baseline"]
    H_cal     = row["DeltaH"]          # già in cal/mol
    err_base  = row["Errore_baseline"]
    name      = f"{campione}_{baseline}"

    if name not in fit_param_dict or np.isnan(H_cal):
        continue

    A_fit, Tm_fit_C, b_fit = fit_param_dict[name]
    Tm_K = Tm_fit_C + 273.15           # per la formula di van’t Hoff
    Cp_Tm = A_fit                      # **** qui ‼️ non più interpolazione

    # ΔH_vH sempre positivo se A_fit > 0
    deltaH_vH = (4 * R * Tm_K**2 * Cp_Tm) / H_cal

    ratio = deltaH_vH / H_cal
    interp = interpreta_cooperativita(ratio)

    records.append({
        "Campione": campione,
        "Baseline": baseline,
        "ΔH_cal (cal/mol)": H_cal,
        "Errore_baseline": err_base,
        "ΔH_vH (cal/mol)": deltaH_vH,
        "HvH / Hcal": ratio,
        "Cooperatività": interp
    })

df_cooperativita = (pd.DataFrame(records)
                    .sort_values(["Campione","Baseline"])
                    .reset_index(drop=True))

display(df_cooperativita.head(12))


,Campione,Baseline,ΔH_cal (cal/mol),Errore_baseline,ΔH_vH (cal/mol),HvH / Hcal,Cooperatività
0,c3,cubica,34159.332539,5971.075017,72047.908788,2.109172,Alta cooperatività (modello a due stati)
1,c3,gradino,46626.665928,5971.075017,61290.852425,1.314502,Alta cooperatività (modello a due stati)
2,c3,lineare,42008.921246,5971.075017,66396.103666,1.580524,Alta cooperatività (modello a due stati)
3,c3,sigmoide,46987.293683,5971.075017,60571.596488,1.289106,Alta cooperatività (modello a due stati)
4,c5,cubica,34292.534704,6481.921626,71019.293573,2.070984,Alta cooperatività (modello a due stati)
5,c5,gradino,47446.113983,6481.921626,61454.146462,1.295241,Alta cooperatività (modello a due stati)
6,c5,lineare,45683.461325,6481.921626,63543.200142,1.390945,Alta cooperatività (modello a due stati)
7,c5,sigmoide,48141.133524,6481.921626,60161.756487,1.249695,Alta cooperatività (modello a due stati)
8,c7,cubica,30715.166830,5665.014682,72774.175733,2.369324,Alta cooperatività (modello a due stati)
9,c7,gradino,40485.064252,5665.014682,64474.660988,1.592554,Alta cooperatività (modello a due stati)


## Dataframe finale Van't Hoff:

In questa ultima sezione consideriamo il modello di Van't Hoff come corretto, avendo già stimato la $T_m$ e il $ \Delta H $, verranno salvati in un file .csv.

La stima dell'incertezza per $T_m$ è la deviazione standard del fit sui dati.

### Valutazione dell’incertezza su *T*<sub>m</sub>

L’errore riportato nella colonna **σ<sub>Tₘ</sub>** è la deviazione standard (1 σ)
associata al parametro *T*<sub>m</sub> del fit con la funzione di van’t Hoff.

`scipy.optimize.curve_fit` restituisce la matrice di covarianza `pcov` dei parametri
ottimizzati; l’elemento `pcov[1, 1]` è la varianza stimata di *T*<sub>m</sub>.
Da qui:

$$
\sigma_{T_m} \;=\; \sqrt{\text{pcov}[1,1]}
$$

Questa stima assume:
1. residui gaussiani indipendenti e identicamente distribuiti,  
2. linearizzazione locale del modello intorno al minimo del χ².  

Per misure nanoDSC con buon rapporto segnale/rumore ciò fornisce
incertezze tipiche di **0,05 – 0,20 °C**.  
Se i residui mostrano forte eteroschedasticità o autocorrelazione la
deviazione standard potrebbe essere sottostimata; in tal caso si può
ricorrere a bootstrap dei residui o a un’analisi Monte Carlo, ma per i
dati presenti la stima da `pcov` è considerata adeguata.


In [6]:
# === DF finale: ΔH_cal, errore baseline, ΔH_vH, Tm e σ_Tm ===============

records_Tm = []      # <-- nome diverso per non sovrascrivere altri 'records'

for name, (A_fit, Tm_fit, b_fit) in fit_param_dict.items():
    campione, baseline = name.split("_", 1)

    # --- valori già calcolati ------------------------------------------------
    row_DH = df_DH[(df_DH["Campione"] == campione) &
                   (df_DH["Baseline"] == baseline)]
    if row_DH.empty:
        continue                     # salta se manca ΔH per quella baseline

    deltaH_val   = row_DH["DeltaH"].iloc[0]
    err_baseline = row_DH["Errore_baseline"].iloc[0]
    sigma_Tm     = fit_sigma_dict.get(name, float("nan"))

    # --- entalpia di van’t Hoff --------------------------------------------
    row_vH = df_cooperativita[(df_cooperativita["Campione"] == campione) &
                              (df_cooperativita["Baseline"] == baseline)]
    deltaH_vH = (row_vH["ΔH_vH (cal/mol)"].iloc[0]
                 if not row_vH.empty else float("nan"))

    records_Tm.append({
        "Campione"          : campione,
        "Baseline"          : baseline,
        "ΔH_cal (cal/mol)"  : deltaH_val,
        "Errore_baseline"   : err_baseline,
        "ΔH_vH (cal/mol)"   : deltaH_vH,
        "Tm (°C)"           : Tm_fit,
        "σ_Tm (°C)"         : sigma_Tm
    })

df_finale = (pd.DataFrame(records_Tm)
             .sort_values(["Campione", "Baseline"])
             .reset_index(drop=True))

display(df_finale)          # o df_finale.to_csv("risultati_finali.csv", index=False)


,Campione,Baseline,ΔH_cal (cal/mol),Errore_baseline,ΔH_vH (cal/mol),Tm (°C),σ_Tm (°C)
0,c3,cubica,34159.332539,5971.075017,72047.908788,65.016854,0.016884
1,c3,gradino,46626.665928,5971.075017,61290.852425,64.995463,0.015927
2,c3,lineare,42008.921246,5971.075017,66396.103666,64.972704,0.020346
3,c3,sigmoide,46987.293683,5971.075017,60571.596488,64.977814,0.011099
4,c5,cubica,34292.534704,6481.921626,71019.293573,65.088637,0.019268
5,c5,gradino,47446.113983,6481.921626,61454.146462,65.039506,0.022468
6,c5,lineare,45683.461325,6481.921626,63543.200142,64.959118,0.022279
7,c5,sigmoide,48141.133524,6481.921626,60161.756487,65.009814,0.014445
8,c7,cubica,30715.166830,5665.014682,72774.175733,64.901687,0.017942
9,c7,gradino,40485.064252,5665.014682,64474.660988,64.971959,0.025803


#### Incertezza su *T*<sub>m</sub>

L’incertezza totale riportata nella colonna **σ<sub>Tₘ</sub> (°C)** combina:

1. **Errore di fit**  
   \(\sigma_{T_m,\text{fit}}\) ricavato dalla radice della varianza del parametro *T*<sub>m</sub>  
   (elemento `pcov[1, 1]`) fornita da `curve_fit`.

2. **Errore termico dello strumento**  
   TA Instruments specifica per il Nano DSC una stabilità di baseline di ±0.028 µW e un controllo termico a stato solido che garantisce precisione < 0.2 °C :contentReference[oaicite:0]{index=0}.  
   Abbiamo quindi aggiunto **±0.2 °C** in quadratura:


$$
\sigma_{T_{m,\text{tot}}} = \left( \sigma_{T_{m,\text{fit}}}^2 + 0.2 \right)^{1/2}
$$

Questa procedura fornisce un errore **assoluto** su *T*<sub>m</sub>, che include sia la precisione numerica del fit sia la tolleranza termica dichiarata dal produttore.


In [7]:
# -- SOMMA IN QUADRATURA dell'errore strumentale ±0.1 °C ---------------
assert "σ_Tm (°C)" in df_finale.columns, "df_finale non contiene la colonna σ_Tm (°C)"

df_finale["σ_Tm (°C)"] = np.sqrt(df_finale["σ_Tm (°C)"]**2 + 0.2**2)

# (facoltativo) arrotonda a due decimali:
# df_finale["σ_Tm (°C)"] = df_finale["σ_Tm (°C)"].round(2)

display(df_finale.head())          # oppure salva: df_finale.to_csv("risultati_finali.csv", index=False)


,Campione,Baseline,ΔH_cal (cal/mol),Errore_baseline,ΔH_vH (cal/mol),Tm (°C),σ_Tm (°C)
0,c3,cubica,34159.332539,5971.075017,72047.908788,65.016854,0.200711
1,c3,gradino,46626.665928,5971.075017,61290.852425,64.995463,0.200633
2,c3,lineare,42008.921246,5971.075017,66396.103666,64.972704,0.201032
3,c3,sigmoide,46987.293683,5971.075017,60571.596488,64.977814,0.200308
4,c5,cubica,34292.534704,6481.921626,71019.293573,65.088637,0.200926


#### Calcolo di entropia e Gibbs libera

L’entropia di transizione è stata ottenuta **integrando la curva DSC**,
secondo l’equazione:

$$
\Delta S(T_m)=\int_{T_i}^{T_f}\frac{C_p(T)}{T}\,dT
$$

mentre l’energia libera di Gibbs a 298 K è:

$$
\Delta G_{298}=\Delta H_{cal}-T_{\mathrm{ref}}\;\Delta S(T_m)
$$

dove $ T_{\mathrm{ref}}=298.15 \text{ K} $.  
Entrambe le relazioni sono riportate da Pagano et al. (METHODS 64, 43–51, 2013) :contentReference[oaicite:2]{index=2}.

**Propagazione degli errori**

* $ σ_{\Delta H} $ – deviazione standard dovuta alla scelta della baseline  
* $ σ_{\Delta S} $ – analoga SD calcolata su ΔS tra le quattro baseline  

$
\sigma_{\Delta G}^2=\sigma_{\Delta H}^2+\bigl(T_{\mathrm{ref}}\;\sigma_{\Delta S}\bigr)^2
$

Le colonne aggiunte ($ ΔS, σ_{ΔS}, ΔG_{298K}, σ_{ΔG, 298K}, ΔG_{310K}, σ_{ΔG,310K} $) sono ora presenti
nel dataframe finale.


In [8]:
# ---------------------------------------------------------------
#  Calcola ΔS(Tm) integrando  Cp(T)/T   e poi ΔG a 298 K.
#  Propaga l’errore dovuto alla scelta della baseline esattamente
#  come hai fatto per ΔH (scarto-tipo tra i modelli di baseline).
# ---------------------------------------------------------------
import numpy as np
import pandas as pd

T_REF1 = 298.15
T_REF2 = 310.15                   # 25 °C
                   # 25 °C
R_to_use = 1.987                 # cal/(mol·K)

# ---- 1. ΔS per ogni singola curva -----------------------------
records_S = []   # Campione-Baseline-ΔS

for name, df in ci_finali_data.items():
    campione, baseline = name.split("_", 1)

    T_K = df["Temperatura"].values + 273.15      # K
    Cp  = df[[c for c in df.columns
              if c.startswith("Cp_finale")][0]].values   # cal/(mol·K)

    # ΔS(Tm): integra Cp/T fra gli stessi limiti usati per ΔH
    S_cal = np.trapezoid(Cp / T_K, T_K)          # cal/(mol·K)

    records_S.append((campione, baseline, S_cal))

df_DS = (pd.DataFrame(records_S, columns=["Campione","Baseline","DeltaS"])
           .sort_values(["Campione","Baseline"]))

# ---- 2. errore_baseline su ΔS  (sd tra modelli) ---------------
sigmaS = (df_DS.groupby("Campione")["DeltaS"]
                .std()
                .rename("Errore_ΔS_baseline"))

df_DS  = df_DS.merge(sigmaS, on="Campione")

# ---- 3. Unisci con ΔH e Tm già calcolati ----------------------
df_finale = (df_finale           # contiene ΔH, σ_H, Tm, σ_Tm
             .merge(df_DS, on=["Campione","Baseline"])
             .rename(columns={"DeltaS":"ΔS (cal/mol·K)",
                              "Errore_ΔS_baseline":"σ_ΔS"})
             .sort_values(["Campione","Baseline"])
             .reset_index(drop=True))

# ---- 4. ΔG a 298 K + propagazione errori ----------------------
H_cal   = df_finale["ΔH_cal (cal/mol)"]
S_cal   = df_finale["ΔS (cal/mol·K)"]
sigma_H = df_finale["Errore_baseline"]
sigma_S = df_finale["σ_ΔS"]

df_finale["ΔG_298K (cal/mol)"] = H_cal - T_REF1 * S_cal
df_finale["σ_ΔG_298K"] = np.sqrt(sigma_H**2 + (T_REF1 * sigma_S)**2)

df_finale["ΔG_310K (cal/mol)"] = H_cal - T_REF2 * S_cal
df_finale["σ_ΔG_310K"] = np.sqrt(sigma_H**2 + (T_REF2 * sigma_S)**2)


display(df_finale.head())


,Campione,Baseline,ΔH_cal (cal/mol),Errore_baseline,ΔH_vH (cal/mol),Tm (°C),σ_Tm (°C),ΔS (cal/mol·K),σ_ΔS,ΔG_298K (cal/mol),σ_ΔG_298K,ΔG_310K (cal/mol),σ_ΔG_310K
0,c3,cubica,34159.332539,5971.075017,72047.908788,65.016854,0.200711,100.930481,17.586936,4066.909518,7946.603114,2855.743742,8087.414277
1,c3,gradino,46626.665928,5971.075017,61290.852425,64.995463,0.200633,137.618081,17.586936,5595.835101,7946.603114,3944.418130,8087.414277
2,c3,lineare,42008.921246,5971.075017,66396.103666,64.972704,0.201032,124.047093,17.586936,5024.280520,7946.603114,3535.715406,8087.414277
3,c3,sigmoide,46987.293683,5971.075017,60571.596488,64.977814,0.200308,138.743492,17.586936,5620.921577,7946.603114,3955.999674,8087.414277
4,c5,cubica,34292.534704,6481.921626,71019.293573,65.088637,0.200926,101.184519,19.118651,4124.370500,8631.794841,2910.156278,8784.990184


### Unità di misura
Al fine di salvare i dati raccolti e confrontarli con altri paper, tutte le entalpie e le energie libere sono riportate in kJ mol⁻¹, mentre le entropie in J mol⁻¹ K⁻¹. I valori originariamente calcolati in cal mol⁻¹ (o cal mol⁻¹ K⁻¹) sono stati convertiti moltiplicando per 4.184 J·cal⁻¹; per le entalpie e ΔG si è poi diviso per 1000 per ottenere kJ.
Gli errori sono stati convertiti con lo stesso fattore, preservando la propagazione corretta delle incertezze.

In [9]:
# ---------------------------------------------------------------
# Converti:
#   ΔH     (cal/mol)   → kJ/mol
#   ΔG     (cal/mol)   → kJ/mol
#   ΔS     (cal/mol·K) → J/mol·K
# Propaga gli errori con lo stesso fattore di conversione.
# ---------------------------------------------------------------
CAL2J  = 4.184
CAL2kJ = CAL2J / 1000        # 0.004184

# 1) entalpia e Gibbs (→ kJ/mol)
for col in ["ΔH_cal (cal/mol)",
            "Errore_baseline",
            "ΔG_298K (cal/mol)",
            "σ_ΔG_298K",
            "ΔG_310K (cal/mol)",
            "σ_ΔG_310K"]:
    if col in df_finale.columns:
        df_finale[col] *= CAL2kJ

# 2) entropia (→ J/mol·K)
for col in ["ΔS (cal/mol·K)", "σ_ΔS"]:
    if col in df_finale.columns:
        df_finale[col] *= CAL2J

# 3) rinomina le intestazioni per chiarezza
df_finale = df_finale.rename(columns={
    "ΔH_cal (cal/mol)"   : "ΔH_cal (kJ/mol)",
    "Errore_baseline"    : "σ_ΔH (kJ/mol)",
    "ΔG_298K (cal/mol)"  : "ΔG_298K (kJ/mol)",
    "σ_ΔG_298K"          : "σ_ΔG_298K (kJ/mol)",
    "ΔG_310K (cal/mol)"  : "ΔG_310K (kJ/mol)",
    "σ_ΔG_310K"          : "σ_ΔG_310K (kJ/mol)",
    "ΔS (cal/mol·K)"     : "ΔS (J/mol·K)",
    "σ_ΔS"               : "σ_ΔS (J/mol·K)"
})

# (facoltativo) arrotonda a 3 decimali per kJ, 1 decimale per J/K
df_finale[["ΔH_cal (kJ/mol)", "σ_ΔH (kJ/mol)",
           "ΔG_298K (kJ/mol)", "σ_ΔG_298K (kJ/mol)",
           "ΔG_310K (kJ/mol)", "σ_ΔG_310K (kJ/mol)"]] = \
    df_finale[["ΔH_cal (kJ/mol)", "σ_ΔH (kJ/mol)",
               "ΔG_298K (kJ/mol)", "σ_ΔG_298K (kJ/mol)",
               "ΔG_310K (kJ/mol)", "σ_ΔG_310K (kJ/mol)"]].round(3)

df_finale[["ΔS (J/mol·K)", "σ_ΔS (J/mol·K)"]] = \
    df_finale[["ΔS (J/mol·K)", "σ_ΔS (J/mol·K)"]].round(1)

display(df_finale.head())
# df_finale.to_csv("risultati_finale_SI.csv", index=False)


,Campione,Baseline,ΔH_cal (kJ/mol),σ_ΔH (kJ/mol),ΔH_vH (cal/mol),Tm (°C),σ_Tm (°C),ΔS (J/mol·K),σ_ΔS (J/mol·K),ΔG_298K (kJ/mol),σ_ΔG_298K (kJ/mol),ΔG_310K (kJ/mol),σ_ΔG_310K (kJ/mol)
0,c3,cubica,142.923,24.983,72047.908788,65.016854,0.200711,422.3,73.6,17.016,33.249,11.948,33.838
1,c3,gradino,195.086,24.983,61290.852425,64.995463,0.200633,575.8,73.6,23.413,33.249,16.503,33.838
2,c3,lineare,175.765,24.983,66396.103666,64.972704,0.201032,519.0,73.6,21.022,33.249,14.793,33.838
3,c3,sigmoide,196.595,24.983,60571.596488,64.977814,0.200308,580.5,73.6,23.518,33.249,16.552,33.838
4,c5,cubica,143.480,27.120,71019.293573,65.088637,0.200926,423.4,80.0,17.256,36.115,12.176,36.756


In [10]:
if "Tel23_risulatai_finalivH.csv" not in os.listdir():
    # Salva il DataFrame finale in un file CSV
    print("Salvataggio del DataFrame finale in '... risultati_finalivH.csv'")
    df_finale.to_csv("Tel23_risultati_finalivH.csv", index=False)

Salvataggio del DataFrame finale in '... risultati_finalivH.csv'
